## [Amazon review](https://www.kaggle.com/snap/amazon-fine-food-reviews)

In [1]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!mkdir -p /content/data
%cd /content/data
!cp /content/drive/MyDrive/data/amazon_review/reviews.csv /content/data

/content/data


In [3]:
import pandas as pd

df = pd.read_csv('/content/data/reviews.csv', error_bad_lines=False) 

In [4]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
summary = df.iloc[0].Summary.split()
text = df.iloc[0].Text.split()

print(len(summary), len(text))

4 48


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [7]:
df.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,568454.000000,568454.000000,568454.00000,568454.000000,5.684540e+05
mean,284227.500000,1.743817,2.22881,4.183199,1.296257e+09
std,164098.679298,7.636513,8.28974,1.310436,4.804331e+07
min,1.000000,0.000000,0.00000,1.000000,9.393408e+08
25%,142114.250000,0.000000,0.00000,4.000000,1.271290e+09
50%,284227.500000,0.000000,1.00000,5.000000,1.311120e+09
75%,426340.750000,2.000000,2.00000,5.000000,1.332720e+09
max,568454.000000,866.000000,923.00000,5.000000,1.351210e+09


In [8]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np
import spacy
from torchtext import data, datasets
from torchtext.data import Field, BucketIterator

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [10]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [11]:
SUMMARY = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True) # batch_first=False
TEXT = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True) # batch_first=False

In [12]:
from torchtext.data import TabularDataset

In [13]:
dataset = TabularDataset(path='/content/data/reviews.csv', format='csv',
        fields=[('Id', None), ('ProductId', None), ('UserId', None), ('ProfileName', None),
                ('HelpfulnessNumerator', None), ('HelpfulnessDenominator', None), ('Score', None), ('Time', None), 
                ('Summary', SUMMARY), ('Text', TEXT)], skip_header=True) 

KeyboardInterrupt: ignored

In [ ]:
sample_data, _ = dataset.split(split_ratio=0.1)
train_data, test_data = sample_data.split(split_ratio=0.7)

In [ ]:
print(type(train_data), type(test_data))

torchtext.data.dataset.Dataset torchtext.data.dataset.Dataset


In [ ]:
print(len(train_data), len(test_data))

39792 17053


In [ ]:
n = 100

In [ ]:
print(train_data[n])

In [ ]:
print(train_data[n].__dict__)

{'Summary': ['no', 'way'], 'Text': ['this', 'was', 'also', 'hazelnut', 'just', 'like', 'the', 'k', '-', 'cups', '.', 'this', 'was', "n't", 'in', 'the', 'description', 'and', 'smelled', 'awful', '.', 'it', 'was', 'wolfgang', 'me', 'crazy', '.']}


In [ ]:
print(type(train_data[n].Summary), len(train_data[n].Summary))
print(type(train_data[n].Text))
print(type(train_data[n+100].Summary), len(train_data[n+100].Summary))
print(type(train_data[n+100].Text))

<class 'list'> 2
<class 'list'>
<class 'list'> 5
<class 'list'>


In [ ]:
Summary = " ".join(train_data[n].Summary)
Text = " ".join(train_data[n].Text)

In [ ]:
print(Summary)
print(Text)

no way
this was also hazelnut just like the k - cups . this was n't in the description and smelled awful . it was wolfgang me crazy .
